# Data Engineering

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

import re
import time

import fundamentus

from google.cloud import storage
import os

import sys
sys.path.append('..')
from scripts.utils import initialize_bucket
import requests
import requests_cache
import logging
import time
from datetime import date

from statsmodels.tsa.holtwinters import SimpleExpSmoothing

2023-04-26 23:42:05,878 [logging.log_init] INFO: LOGLEVEL=INFO


In [2]:
pd.set_option("display.max_columns", 200)

In [3]:
credentials_path = '../datascience-capstone-project-05b1642f45c3.json'

client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

## 1 - Data Collection

For the data gathering, two repositories on Github were combined, so that the historical data could be scraped from the web. This Raw data is beeing stored in the cloud (GCP) and are the main data used for the project.

Repositories utilized:
* https://github.com/mv/fundamentus-api
* https://github.com/Victorcorcos/bovespa-winner

In [22]:
def perc_to_float(val):
    """
    Percent to float
      - replace string in pt-br to float
      - from '45,56%' to 0.4556
    Input:
        (DataFrame, column_name)
    """

    res = val
    res = res.replace( to_replace=r'[%]', value='' , regex=True )
    res = res.replace( to_replace=r'[.]', value='' , regex=True )
    res = res.replace( to_replace=r'[,]', value='.', regex=True )
    res = res.astype(float) / 100

    return res

def _rename_cols(data):
    """
    Rename columns in DataFrame
      - use a valid Python identifier
      - so each column can be a DataFrame property
      - Example:
          df.pl > 0
    """

    df2 = pd.DataFrame()

    ## Fix: rename columns
    df2['cotacao'  ] = data['Cotação'          ]
    df2['pl'       ] = data['P/L'              ]
    df2['pvp'      ] = data['P/VP'             ]
    df2['psr'      ] = data['PSR'              ]
    df2['dy'       ] = data['Div.Yield'        ]
    df2['pa'       ] = data['P/Ativo'          ]
    df2['pcg'      ] = data['P/Cap.Giro'       ]
    df2['pebit'    ] = data['P/EBIT'           ]
    df2['pacl'     ] = data['P/Ativ Circ.Liq'  ]
    # df2['evebit'   ] = data['EV/EBIT'          ]
    # df2['evebitda' ] = data['EV/EBITDA'        ] ##
    df2['mrgebit'  ] = data['Mrg Ebit'         ]
    df2['mrgliq'   ] = data['Mrg. Líq.'        ]
    df2['roic'     ] = data['ROIC'             ]
    df2['roe'      ] = data['ROE'              ]
    df2['liqc'     ] = data['Liq. Corr.'       ]
    df2['liq2m'    ] = data['Liq.2meses'       ]
    df2['patrliq'  ] = data['Patrim. Líq'      ]
    df2['divbpatr' ] = data['Dív.Brut/ Patrim.']
    df2['c5y'      ] = data['Cresc. Rec.5a'    ]

    return df2

def get_resultado_raw(url):
    """
    Get data from fundamentus:
      URL:
        http://fundamentus.com.br/resultado.php
    RAW:
      DataFrame preserves original HTML header names
    Output:
      DataFrame
    """

    ##
    ## Busca avançada por empresa
    ##
    # url = 'http://www.fundamentus.com.br/resultado.php'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01',
           'Accept-Encoding': 'gzip, deflate',
           }

    with requests_cache.enabled():
        content = requests.get(url, headers=hdr)

        if content.from_cache:
            logging.debug('.../resultado.php: [CACHED]')
        else: # pragma: no cover
            logging.debug('.../resultado.php: sleeping...')
            time.sleep(.500) # 500 ms


    ## parse + load
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]

    ## Fix: percent string
    df['Div.Yield']     = perc_to_float( df['Div.Yield']     )
    df['Mrg Ebit']      = perc_to_float( df['Mrg Ebit']      )
    df['Mrg. Líq.']     = perc_to_float( df['Mrg. Líq.']     )
    df['ROIC']          = perc_to_float( df['ROIC']          )
    df['ROE']           = perc_to_float( df['ROE']           )
    df['Cresc. Rec.5a'] = perc_to_float( df['Cresc. Rec.5a'] )

    ## index by 'Papel', instead of 'int'
    df.index = df['Papel']
    df.drop('Papel', axis='columns', inplace=True)
    df.sort_index(inplace=True)

    ## naming
    df.name = 'Fundamentus: HTML names'
    df.columns.name = 'Multiples'
    df.index.name = 'papel'

    ## return sorted by 'papel'
    return df


def get_resultado(url):
    """
    Data from fundamentus, fixing header names.
      URL:
        given from the user
      Obs:
        DataFrame uses short header names
    Output:
      DataFrame
    """

    ## get RAW data
    data1 = get_resultado_raw(url)

    ## rename!
    data2 = _rename_cols(data1)

    ## metadata
    data2.name = 'Fundamentus: short names'
    data2.columns.name = 'Multiples'
    data2.index.name = 'papel'

    ## remove duplicates
#   df = data2.drop_duplicates(subset=['cotacao','pl','pvp'], keep='last')
    df = data2.drop_duplicates(keep='first')

    return df


In [24]:
def get_windows(max, min, window=5, step=1):

    try:
        maxes = list(range(min+window,max+1, step))
        mins = list(range(min, max+1-window, step))
    except:
        print(f'error 1!!! min {maxes}, max {maxes}')
    sequences = []
    if len(maxes)==len(mins):
        for i in range(len(maxes)):
            sequence = list(range(mins[i],maxes[i]+1,step))
            sequences.append(sequence)
        return sequences
    else:
        print(f'error 2!!! min {maxes}, max {maxes}')

### Get historical data

In [75]:
def get_dates(urls):
    dates = []
    for value in urls:
        str_1 = re.split('/web/',value)[1]
        str_2 = re.split('/http',str_1)[0]
        str_date = f'{str_2[:4]}-{str_2[4:6]}-{str_2[6:8]}'
        dates.append(pd.to_datetime(str_date))
    return dates



def ingest_data(file, path, time_to_sleep = 1, store_locally=False):
    
    filename = path+file
    urls = pd.read_csv(filename).sort_values(by='urls')

    dates = get_dates(urls['urls'].values)

    date_url = list(zip(dates, urls['urls'].values))

    df_full = pd.DataFrame()

    year = dates[0].year

    for date, url in date_url[:]:
        print(date,url)

        df = get_resultado(url)
        df['date'] = date
        df['year'] = date.year

        df_full = pd.concat([df_full,df])

        if year == date.year:
            continue
        elif store_locally:
           df_full.loc[df_full['year']==year].to_csv(f'../data/01_trusted/{str(date.year)}.csv')
           year = date.year

        time.sleep(time_to_sleep) 
           

    return df_full.reset_index()


def get_detailed_ticker_data(tickers):

    for ticker in tickers:
        try:
            df = fundamentus.get_ticker(ticker)
            df_tickers = pd.concat([df_tickers,df])
        except:
            print(f'fail ticker {ticker}')
        time.sleep(3)
    return df_tickers


In [92]:
# df = ingest_data('urls.csv', '../data/00_raw/', time_to_sleep=3, store_locally=False)

2008-06-13 00:00:00 https://web.archive.org/web/20080613050809/http://www.fundamentus.com.br/resultado.php
2008-07-02 00:00:00 https://web.archive.org/web/20080702065244/http://www.fundamentus.com.br/resultado.php
2008-10-20 00:00:00 https://web.archive.org/web/20081020204401/http://www.fundamentus.com.br/resultado.php
2008-12-01 00:00:00 https://web.archive.org/web/20081201055839/http://www.fundamentus.com.br/resultado.php
2008-12-05 00:00:00 https://web.archive.org/web/20081205110450/http://www.fundamentus.com.br/resultado.php
2009-02-03 00:00:00 https://web.archive.org/web/20090203013110/http://www.fundamentus.com.br/resultado.php
2009-03-11 00:00:00 https://web.archive.org/web/20090311122726/http://www.fundamentus.com.br/resultado.php
2009-04-12 00:00:00 https://web.archive.org/web/20090412044033/http://www.fundamentus.com.br/resultado.php
2009-07-03 00:00:00 https://web.archive.org/web/20090703215949/http://www.fundamentus.com.br/resultado.php
2009-08-21 00:00:00 https://web.archi

In [93]:
df.head()

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,date,year
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008-06-13,2008
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008-06-13,2008
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,8.33,0.2182,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008-06-13,2008
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,25.25,0.2520,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008-06-13,2008
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,6.54,0.2352,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008-06-13,2008


### Get detailed information about each ticker

In [94]:
tickers = df['papel'].unique()

# df_tickers = get_detailed_ticker_data(tickers)

In [99]:
df_tickers.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,Valor_de_mercado,Valor_da_firma,Ult_balanco_processado,Nro_Acoes,PL,PVP,PEBIT,PSR,PAtivos,PCap_Giro,PAtiv_Circ_Liq,Div_Yield,EV_EBITDA,EV_EBIT,Cres_Rec_5a,LPA,VPA,Marg_Bruta,Marg_EBIT,Marg_Liquida,EBIT_Ativo,ROIC,ROE,Liquidez_Corr,Div_Br_Patrim,Giro_Ativos,Ativo,Cart_de_Credito,Depositos,Patrim_Liq,Result_Int_Financ_12m,Rec_Servicos_12m,Lucro_Liquido_12m,Result_Int_Financ_3m,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.00,0,0,-,2022-12-31,226090000,000,000,-,-,-,-,-,0.0%,-,-,27.5%,000,000,-,-,0.0%,0.0%,-,15.4%,-,-,-,56768500000,0,0,5205390000,1457840000,367979000,800234000,344592000,90254000,197272000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,16.37,2023-04-24,14.47,22.30,12612700,3701100000,-,2022-12-31,226090000,463,071,-,-,-,-,-,6.6%,-,-,27.5%,354,2302,-,-,0.0%,0.0%,-,15.4%,-,-,-,56768500000,0,0,5205390000,1457840000,367979000,800234000,344592000,90254000,197272000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.00,0,515088000,868691000,2010-03-31,104906000,-21480,176,1996,205,053,198,-275,0.0%,3367,3367,16.4%,-002,278,32.0%,10.3%,-1.0%,2.6%,2.8%,-0.8%,209,131,026,978287000,NaN,NaN,292060000,NaN,NaN,-2398000,NaN,NaN,-766000,29556000,498222000,383159000,353603000,250657000,25802000,67990000,18057000
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.00,0,7080470000,6437160000,2007-12-31,74320000,879,228,720,169,144,640,2077,0.0%,654,654,13.3%,1084,4179,32.8%,23.5%,19.3%,20.0%,26.9%,25.9%,205,009,085,4922360000,NaN,NaN,3105800000,NaN,NaN,805922000,NaN,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.00,0,7040330000,6397030000,2007-12-31,74320000,874,227,716,168,143,637,2065,0.0%,650,650,13.3%,1084,4179,32.8%,23.5%,19.3%,20.0%,26.9%,25.9%,205,009,085,4922360000,NaN,NaN,3105800000,NaN,NaN,805922000,NaN,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000


### Storing the raw data into Google Cloud

In [98]:
# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('01_raw/fundamentus_historical_raw.csv')
blob.upload_from_string(df.to_csv(), 'text/csv')
# # upload raw fundamentalist data to Google Cloud Storage
# blob = bucket.blob('01_raw/fundamentus_tickers_raw.csv')
# blob.upload_from_string(df_tickers.to_csv(), 'text/csv')

## 2 - Data Cleansing

### Load data

In [62]:
path = "gs://storage-barsianize/01_raw/fundamentus_historical_raw.csv"
df =  pd.read_csv(path, index_col=0)

path = "gs://storage-barsianize/01_raw/fundamentus_tickers_raw.csv"
df_tickers = pd.read_csv(path, index_col=0)

#### Interpolate the data to get the daily data

In [5]:
df.head()

,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,date,year
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008-06-13,2008
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008-06-13,2008
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,8.33,0.2182,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008-06-13,2008
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,25.25,0.2520,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008-06-13,2008
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,6.54,0.2352,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008-06-13,2008


In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
columns_to_spline = ['cotacao',
                     'pl',
                     'pvp',
                     'psr',
                     'dy',
                     'pa',
                     'pcg',
                     'pebit',
                     'pacl',
                     'evebit',
                     'mrgebit',
                     'mrgliq',
                     'roic',
                     'roe',
                     'liqc',
                     'liq2m',
                     'patrliq',
                     'divbpatr',
                     'c5y']

columns_to_first = ['papel']



In [8]:

def create_daily_dataset(data, date_col):
    data_daily = pd.DataFrame(pd.date_range(data[date_col].min(), data[date_col].max(), freq='D'), 
                     columns=['days']).merge(data, how='left', left_on='days', right_on=date_col)
    return data_daily


def interpolate_data(data, cols_first, cols_spline, date_col):

    data = create_daily_dataset(data, date_col)

    columns = data.columns
    for col in columns:
        if col in cols_first:
            data[col] = data[col].fillna(data[col][0])
        elif col in cols_spline:
            data[col] = data[col].interpolate(method='spline', order=2)
        else:
            continue
    return data

def get_daily_data_per_ticker(data, ticker_col, cols_first, cols_spline, date_col):
    tickers = data[ticker_col].unique()
    # tickers = ['BBAS3','ITUB4','PETR4','CIEL3']

    daily_dataframes = []

    for ticker in tickers:
        print(ticker)
        try:
            daily_dataframes.append(
                    interpolate_data(data[data[ticker_col]==ticker], cols_first, cols_spline, date_col)
                )
        except:
                continue
    
    daily_data = pd.concat(daily_dataframes)

    return daily_data



In [9]:
df_daily = get_daily_data_per_ticker(df, 'papel', columns_to_first, columns_to_spline, 'date')

ABCB3
ABCB4
ABNB3


c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


ABYA3
ACES3
ACES4
ACGU3
AEDU11
AELP3
AESL3


c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


AFLU3
AFLU5
AGEN11
AGIN3


...

TRAD3
TTEN3
BLUT3
BLUT4
CLSA3
DXCO3
GETT3
GETT4
KRSA3
ONCO3
RAIZ4
SYNE3
VIIA3
VITT3


c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smo

VVEO3
IGTI11
IGTI3
MEGA3
PORT3
VBBR3
AURE3


c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\Pontello\anaconda3\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smo

LAND3
NEXP3
CSUD3
DMFN3
ZAMP3
VSTE3
WIZC3


In [10]:
df_daily['date'] = df_daily['days']
df_daily['year'] = df_daily['days'].dt.year

print(df_daily.shape)

(4534343, 23)


Upload results

In [ ]:
# path_local = '../data/01_trusted/'
# filename = 'daily_data.csv'
# df_daily.to_csv(path_local + filename)

In [27]:
# path_local = '../data/01_trusted/'
# filename = 'daily_data.parquet'
# blob_name = '02_trusted/'

# # save the DataFrame as a parquet file
# df_daily.to_parquet(path_local + filename)

# # upload the parquet file to Google Cloud Storage
# blob = bucket.blob(blob_name + filename)
# blob._chunk_size = 8388608
# blob.upload_from_filename(path_local + filename, num_retries=10, )

#### Feature engineering for each corporation 

In [ ]:
path = "gs://storage-barsianize/02_trusted/daily_data.parquet"
df_test =  pd.read_parquet(path)

# df_daily = pd.read_parquet('../data/01_trusted/daily_data.parquet')

-	Create a BESST feature
-	Create a feature about the size of the company
Patrim_Liq, Lucro_Liquido_12m, Ativo, Valor_de_mercado, Setor, Subsetor	

In [63]:
def create_numerical_categories(data, cols, n):
    
    for col in cols:
        data[col + '_category'] = pd.qcut(df_tickers['Patrim_Liq'], 
                                          q=n, duplicates='drop', 
                                          labels=list(range(n)))
        
    return data


In [64]:
df_tickers = create_numerical_categories(df_tickers, ['Valor_de_mercado'], 5)

In [70]:
df_tickers = df_tickers[~(pd.to_datetime(df_tickers['Data_ult_cot']).dt.year<2008)]

In [85]:
df_tickers = create_numerical_categories(df_tickers, ['Valor_de_mercado','Lucro_Liquido_12m','Receita_Liquida_12m','Patrim_Liq'], 5)

In [90]:
(df_tickers['Valor_de_mercado_category'].astype(int) - df_tickers['Receita_Liquida_12m_category'].astype(int)).value_counts()

0    824
dtype: int64

In [86]:
df_tickers[df_tickers['Valor_de_mercado_category']==3].iloc[20:80]

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,Valor_de_mercado,Valor_da_firma,Ult_balanco_processado,Nro_Acoes,PL,PVP,PEBIT,PSR,PAtivos,PCap_Giro,PAtiv_Circ_Liq,Div_Yield,EV_EBITDA,EV_EBIT,Cres_Rec_5a,LPA,VPA,Marg_Bruta,Marg_EBIT,Marg_Liquida,EBIT_Ativo,ROIC,ROE,Liquidez_Corr,Div_Br_Patrim,Giro_Ativos,Ativo,Cart_de_Credito,Depositos,Patrim_Liq,Result_Int_Financ_12m,Rec_Servicos_12m,Lucro_Liquido_12m,Result_Int_Financ_3m,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m,Valor_de_mercado_category,Lucro_Liquido_12m_category,Receita_Liquida_12m_category,Patrim_Liq_category
CLSC5,CLSC5,PNA N2,CELESC PNA N2,Energia Elétrica,Energia Elétrica,39.01,2011-04-25,0.00,0.00,0,1504690000,3013280000,2022-12-31,38572000,278,52,227,015,013,-1909,-030,0.0%,321,455,9.4%,1401,7475,14.3%,6.6%,5.4%,5.7%,6.8%,18.7%,098,085,087,11628100000,NaN,NaN,2883180000,NaN,NaN,540562000,NaN,NaN,103372000,9.406840e+08,3.678620e+09,2.449270e+09,1.508580e+09,1.008280e+10,6.622120e+08,2.581270e+09,1.205370e+08,3,3,3,3
CLSC6,CLSC6,PNB N2,CELESC PNB N2,Energia Elétrica,Energia Elétrica,0.00,NaN,0.00,0.00,0,0,-,2022-12-31,38572000,0,0,-,-,-,-,-,0.0%,-,-,9.4%,0,0,-,-,-,0.0%,-,-,-,-,-,11628100000,NaN,NaN,2883180000,NaN,NaN,0,NaN,NaN,0,9.406840e+08,3.678620e+09,2.449270e+09,1.508580e+09,1.008280e+10,6.622120e+08,2.581270e+09,1.205370e+08,3,3,3,3
COCE3,COCE3,ON,CIA ENERG CEARA - COELCE ON,Energia Elétrica,Energia Elétrica,57.00,2023-04-20,57.00,57.24,292,4437730000,8827390000,2022-12-31,77855000,693,112,366,052,036,-338,-081,3.1%,545,728,15.6%,823,5082,16.8%,14.2%,7.5%,9.8%,11.4%,16.2%,068,118,070,12309400000,NaN,NaN,3956900000,NaN,NaN,640570000,NaN,NaN,251017000,2.694580e+08,2.849930e+09,4.659110e+09,4.389650e+09,8.568300e+09,1.212430e+09,2.222440e+09,3.626220e+08,3,3,3,3
COCE5,COCE5,PNA,CIA ENERG CEARA - COELCE PNA,Energia Elétrica,Energia Elétrica,51.09,2023-04-24,37.00,52.66,2955840,3977610000,8367270000,2022-12-31,77855000,621,101,328,046,032,-303,-072,3.4%,516,690,15.6%,823,5082,16.8%,14.2%,7.5%,9.8%,11.4%,16.2%,068,118,070,12309400000,NaN,NaN,3956900000,NaN,NaN,640570000,NaN,NaN,251017000,2.694580e+08,2.849930e+09,4.659110e+09,4.389650e+09,8.568300e+09,1.212430e+09,2.222440e+09,3.626220e+08,3,3,3,3
COCE6,COCE6,PNB,CIA ENERG CEARA - COELCE PNB,Energia Elétrica,Energia Elétrica,35.00,2012-03-20,0.00,0.00,0,2724920000,7114580000,2022-12-31,77855000,425,69,225,032,022,-208,-050,0.0%,439,587,15.6%,823,5082,16.8%,14.2%,7.5%,9.8%,11.4%,16.2%,068,118,070,12309400000,NaN,NaN,3956900000,NaN,NaN,640570000,NaN,NaN,251017000,2.694580e+08,2.849930e+09,4.659110e+09,4.389650e+09,8.568300e+09,1.212430e+09,2.222440e+09,3.626220e+08,3,3,3,3
CRUZ3,CRUZ3,ON,SOUZA CRUZ ON,NaN,NaN,27.59,2015-11-30,0.00,0.00,0,42169900000,42192400000,2015-09-30,1528450000,2531,2246,2074,661,718,6203,12493,0.0%,1927,2075,3.0%,109,123,61.1%,31.9%,26.1%,34.6%,46.8%,88.8%,119,063,109,5869180000,NaN,NaN,1877350000,NaN,NaN,1666240000,NaN,NaN,344582000,1.160400e+09,4.329390e+09,1.182820e+09,2.241700e+07,6.375550e+09,2.033470e+09,1.543160e+09,4.141720e+08,3,3,3,3
DAYC4,DAYC4,PN N1,Daycoval PN N1,Intermediários Financeiros,Bancos,8.90,2016-08-23,0.00,0.00,0,2149030000,-,2017-12-31,241464000,412,71,-,-,-,-,-,0.0%,-,-,8.1%,216,1246,-,-,0.0%,0.0%,-,17.3%,-,-,-,24231000000,0.0,0.0,3009030000,0.0,0.0,521474000,0.0,0.0,129961000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,3,3
DURA3,DURA3,ON,DURATEX ON,Madeira e Papel,Madeira,31.90,2009-10-07,0.00,0.00,0,4041570000,4571940000,2009-06-30,126695000,1662,226,1029,222,124,1752,-1066,0.0%,1164,1164,10.9%,192,1413,38.7%,21.6%,13.4%,12.1%,14.4%,13.6%,127,052,056,3249020000,NaN,NaN,1789980000,NaN,NaN,243237000,NaN,NaN,44969000,4.000660e+08,1.077800e+09,9.304380e+08,5.303720e+08,1.822360e+09,3.929320e+08,4.132790e+08,8.264700e+07,3,3,3,3
DURA4,DURA4,PN,DURATEX PN,Madeira e Papel,Madeira,31.39,2009-10-07,0.00,0.00,0,3976960000,4507330000,2009-06-30,1266

In [13]:
# info_papeis = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot']
info_tickers = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot']
               
df_tickers_clean = df_tickers[info_tickers]

df_tickers_clean = df_tickers_clean.dropna()
df_tickers_clean = df_tickers_clean.drop_duplicates()

In [14]:
df_tickers_clean['Data_ult_cot'] = pd.to_datetime(df_tickers_clean['Data_ult_cot'])

In [15]:
df_completed = df_tickers_clean.merge(df, how='left', left_index=True, right_on='papel').drop_duplicates()

In [16]:
df_completed.isna().mean().sort_values(ascending=False)

Papel           0.0
Tipo            0.0
date            0.0
c5y             0.0
divbpatr        0.0
patrliq         0.0
liq2m           0.0
liqc            0.0
roe             0.0
roic            0.0
mrgliq          0.0
mrgebit         0.0
evebit          0.0
pacl            0.0
pebit           0.0
pcg             0.0
pa              0.0
dy              0.0
psr             0.0
pvp             0.0
pl              0.0
cotacao         0.0
papel           0.0
Data_ult_cot    0.0
Subsetor        0.0
Setor           0.0
Empresa         0.0
year            0.0
dtype: float64

In [17]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('02_trusted/df_completed.csv')
blob.upload_from_string(df_completed.to_csv(), 'text/csv',)

## 3 - Data Preparation

In [18]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

In [24]:
path = "gs://storage-barsianize/trusted/df_completed.csv"
df_completed =  pd.read_csv(path, index_col=0)
df_completed['Data_ult_cot'] = pd.to_datetime(df_completed['Data_ult_cot'])
df_completed['date'] = pd.to_datetime(df_completed['date'])

In [31]:
def interpolate_data(serie, time_begin, time_ends):
    # define the time range for the spline
    time_range = pd.date_range(time_begin, time_ends, freq='D')

    # create an interpolation function using a cubic spline
    f = interp1d(serie.index.astype(int), serie.values, kind='cubic')

    # evaluate the interpolation function at the new time points
    new_data = pd.Series(f(time_range.astype(int)), index=time_range)

    return new_data

In [28]:
df_completed.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Data_ult_cot,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,1.107010e+09,0.0,0.0000,2008,2008-06-13
736,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1553,0.0,0.0,1.158760e+09,0.0,0.0000,2009,2009-01-23
1478,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1016,0.0,0.0,1.182180e+09,0.0,-0.1233,2010,2010-01-15
2237,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1521,0.0,0.0,1.312080e+09,0.0,0.1043,2011,2011-01-13
3020,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1571,0.0,0.0,1.460150e+09,0.0,0.1272,2012,2012-01-06


In [29]:
df_bb = df_completed.loc[df_completed['Papel']=='BBAS3']

In [34]:
df_bb['date'].max()

Timestamp('2023-03-24 00:00:00')

In [41]:
df_bb = pd.DataFrame(pd.date_range(df_bb['date'].min(), df_bb['date'].max(), freq='D'), columns=['days']).merge(df_bb, how='left', left_on='days', right_on='date')

In [42]:
df_bb.head()

,days,Papel,Tipo,Empresa,Setor,Subsetor,Data_ult_cot,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year,date
0,2008-06-13,BBAS3,ON,BANCO DO BRASIL S.A. ON,Intermediários Financeiros,Bancos,2023-01-04,BBAS3,29.61,12.55,2.96,0.0,0.0273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.236,0.0,123934000.0,2.540680e+10,0.0,0.0972,2008.0,2008-06-13
1,2008-06-14,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,2008-06-15,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2008-06-16,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,2008-06-17,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [35]:
df_bb[['date']].interpolate(method='time',)

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got time

In [32]:
interpolate_data(df_bb['cotacao'],df_bb['date'].min(), df_bb['date'].max())

ValueError: A value in x_new is above the interpolation range.

In [25]:
df_completed.columns

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'Data_ult_cot',
       'papel', 'cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
       'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
       'patrliq', 'divbpatr', 'c5y', 'year', 'date'],
      dtype='object')

In [26]:
df_completed['Data_ult_cot_year'] = df_completed['Data_ult_cot'].dt.year

In [27]:
df_actual = df_completed[df_completed['Data_ult_cot_year'] == pd.to_datetime(date.today()).year]

In [28]:
tickers = df_actual['Papel'].unique()

In [29]:
tickers

array(['ABCB4', 'AGRO3', 'ALPA3', 'ALPA4', 'AZEV3', 'AZEV4', 'BAHI3',
       'BALM3', 'BALM4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BDLL4',
       'BEEF3', 'BEES3', 'BGIP4', 'BIOM3', 'BMEB3', 'BMEB4', 'BMIN3',
       'BMIN4', 'BMKS3', 'BOBR4', 'BRAP3', 'BRAP4', 'BRGE11', 'BRGE3',
       'BRIV3', 'BRIV4', 'BRKM3', 'BRKM5', 'BRML3', 'BRSR3', 'BRSR5',
       'BRSR6', 'CALI3', 'CAMB3', 'CBEE3', 'CCRO3', 'CEBR3', 'CEBR5',
       'CEBR6', 'CEDO4', 'CEEB3', 'CGAS3', 'CGAS5', 'CGRA3', 'CGRA4',
       'CLSC3', 'CMIG3', 'CMIG4', 'COCE5', 'CPFE3', 'CPLE3', 'CPLE6',
       'CRIV3', 'CRIV4', 'CSAB3', 'CSAB4', 'CSAN3', 'CSMG3', 'CSNA3',
       'CSRN3', 'CSRN5', 'CSRN6', 'CTKA4', 'CTNM4', 'CTSA3', 'CTSA4',
       'CYRE3', 'DASA3', 'DOHL4', 'EALT3', 'EALT4', 'EKTR3', 'EKTR4',
       'ELET3', 'ELET6', 'EMAE4', 'EMBR3', 'ENBR3', 'ENGI3', 'ENGI4',
       'EQTL3', 'ESTR4', 'ETER3', 'EUCA3', 'EUCA4', 'EVEN3', 'EZTC3',
       'FESA3', 'FESA4', 'FHER3', 'GEPA3', 'GEPA4', 'GFSA3', 'GGBR3',
       'GGBR4', 'GO

In [30]:
stock_info = ['Papel','Tipo','Empresa','Setor','Subsetor']

indicators = ['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
              'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
              'patrliq', 'divbpatr', 'c5y','date','year']

In [31]:
df_indicators = df_actual[np.isin(df_actual['Papel'],tickers)][stock_info + indicators]

In [32]:
data = df_actual[df_actual['Papel']==tickers[2]].sort_values('year', ascending=False)

max = data['year'].max()
min = data['year'].min()
windows = get_windows(max, min, window=5, step=1)

### Get relative information for each sector

In [33]:
cols_to_rel = ['mrgliq','liq2m','patrliq','mrgebit','c5y']

def relativize(data, cols, sector_col, window):
    """
    Relativize the values in a DataFrame by dividing them by the corresponding values in a reference DataFrame.

    Parameters
    ----------
    data : pandas.DataFrame
        The DataFrame to relativize.
    cols : list of str
        The column names to relativize.
    sector_col : str
        The name of the column containing the sector information.

    Returns
    -------
    pandas.DataFrame
        A copy of the input DataFrame `data` with the specified columns relativized.
    """
    data_out = data.copy()
    sectors = data_out[sector_col].unique()
    max_window = np.max(window)

    ref_data = data_out.groupby(sector_col)[cols].mean()
    
    
    data_out = data_out[data_out['year']==max_window]
    for sector in sectors:
        sector_data = data_out.loc[data_out[sector_col] == sector]
        for col in cols:
            sector_data[col] /= ref_data.at[sector, col]
        data_out.loc[data_out[sector_col] == sector] = sector_data
    # data_out = data_out[[sector_col] + cols + ['year','Papel']]

    ref_data.columns = [col+'_field' for col in ref_data.columns]
    ref_data['year'] = max_window
    
    return ref_data, data_out

In [34]:
max = 2023
min = 2008

windows = get_windows(max, min, window=5, step=1)

In [35]:
databases = []
for window in windows:
    
    ref_data, data_out = relativize(df_actual[np.isin(df_actual['year'],window)], cols_to_rel, 'Setor',window)
    database = data_out.merge(ref_data, how='left', left_on='Setor',right_index=True)
    databases.append(database)

database_full = pd.concat(databases)

database_full['year'] = database_full['year_x'].astype(int)
database_full.drop(['year_x','year_y'],axis=1, inplace=True)

C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\1993705781.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_data[col] /= ref_data.at[sector, col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\1993705781.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_data[col] /= ref_data.at[sector, col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\1993705781.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [36]:
import requests
import pandas as pd

def get_bcb_data(codes, start_date, end_date, output_format='json'):
    """
    Get macroeconomic data from the Brazilian Central Bank's API.

    Args:
        codes (list): List of series codes to retrieve. See the API documentation for available codes.
        start_date (str): Start date in format 'dd/mm/yyyy'.
        end_date (str): End date in format 'dd/mm/yyyy'.
        output_format (str, optional): Output format, either 'json' or 'csv'. Defaults to 'json'.

    Returns:
        pandas.DataFrame: DataFrame with the requested series data.

    Raises:
        ValueError: If the output format is invalid.
        requests.exceptions.RequestException: If the API request fails.

    Example:
        Get the values of the crude oil production, commodities, dollar, euro, IPCA, IGPM, and Selic series from January 1st, 2021 to December 31st, 2021 in JSON format:

        >>> codes = [13522, 13521, 4390, 189, 11, 1178]
        >>> start_date = '01/01/2021'
        >>> end_date = '31/12/2021'
        >>> output_format = 'json'
        >>> df = get_bcb_data(codes, start_date, end_date, output_format)

    """
    # Define the base URL of the API
    url_base = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados"

    # Define the output format parameter
    if output_format not in ['json', 'csv']:
        raise ValueError("Invalid output format. Must be 'json' or 'csv'.")
    formato = output_format

    # Create a dictionary to store the series DataFrames
    dataframes = {}

    # Get the data for each series and store it in a DataFrame
    for code_name, code in codes.items():
        # Build the complete URL with the defined parameters
        url = url_base.format(code) + f"?formato={formato}&dataInicial={start_date}&dataFinal={end_date}"
        # Make the API request
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response to a pandas DataFrame and set the column name to the series code
            try:
                df = pd.read_json(response.text)
            except:
                try:
                    df = pd.read_xml(response.text)
                except:
                    continue
            col_name = code_name
            df = df.rename(columns={"valor": col_name})
            # Set the DataFrame index to the date
            df = df.set_index("data")
            # Store the DataFrame in the dictionary
            dataframes[col_name] = df[col_name]
        else:
            raise requests.exceptions.RequestException(f"Error getting data. HTTP status code: {response.status_code}")

    # Combine the DataFrames for each series into a single DataFrame
    final_df = pd.concat(dataframes.values(), axis=1)

    return final_df

In [37]:
# https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

codes = {
    'preco_do_petroleo': 4390,
    'preco_do_minerio_de_ferro': 25521,
    'indice_da_industria': 24369,
    'indice_do_agro': 24368,
    'dolar_comercial': 1,
    'euro': 21619,
    'ibovespa': 23686,
    'pib': 21920,
    'pib_dolarizado': 22786,
    'igpm': 189,
    'ipca': 433,
    'selic': 11
}
start_date = "01/01/2008"
end_date = "28/02/2023"
output_format='json'

final_df = get_bcb_data(codes, start_date, end_date, output_format=output_format)

In [38]:
final_df

,preco_do_petroleo,indice_da_industria,dolar_comercial,euro,ibovespa,pib_dolarizado,igpm,ipca,selic
data,,,,,,,,,
01/01/2008,0.93,NaN,NaN,NaN,267227.1,1183.3,1.09,0.54,NaN
01/02/2008,0.80,NaN,1.7451,2.58659,NaN,771.3,0.53,0.49,0.042065
01/03/2008,0.84,NaN,NaN,NaN,NaN,1107.8,0.74,0.48,NaN
01/04/2008,0.90,NaN,1.7534,2.73574,NaN,1005.5,0.69,0.55,0.042065
01/05/2008,0.88,NaN,NaN,NaN,NaN,838.4,1.61,0.79,NaN
...,...,...,...,...,...,...,...,...,...
22/02/2023,NaN,NaN,5.1730,5.49790,NaN,NaN,NaN,NaN,0.050788
23/02/2023,NaN,NaN,5.1330,5.44050,NaN,NaN,NaN,NaN,0.050788
24/02/2023,NaN,NaN,5.1791,5.46080,NaN,NaN,NaN,NaN,0.050788


In [39]:
final_df = final_df.sort_values(by='data').reset_index()
final_df['data'] = pd.to_datetime(final_df['data'])
final_df['year'] = final_df['data'].dt.year
final_df['month'] = final_df['data'].dt.month
final_df = final_df.groupby(pd.Grouper(key='data', freq='Y',origin=0, label='left')).first()\
                    .fillna(0).reset_index()

bc_data_columns = final_df.columns
database_full['date'] = pd.to_datetime(database_full['date'])
database_full['year'] = database_full['date'].dt.year
database_full = database_full.copy()
database_full = database_full.merge(final_df, how='left', on='year')

C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\436442326.py:2: UserWarning: Parsing '13/01/2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  final_df['data'] = pd.to_datetime(final_df['data'])
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\436442326.py:2: UserWarning: Parsing '13/01/2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  final_df['data'] = pd.to_datetime(final_df['data'])
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\436442326.py:2: UserWarning: Parsing '13/01/2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  final_df['data'] = pd.to_datetime(final_df['data'])
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\436442326.py:2: UserWarning: Parsing '13/01/2012' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  final_df['data'] = pd.to_datetim

In [40]:
database_full.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Data_ult_cot,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,date,Data_ult_cot_year,mrgliq_field,liq2m_field,patrliq_field,mrgebit_field,c5y_field,year,data,preco_do_petroleo,indice_da_industria,dolar_comercial,euro,ibovespa,pib_dolarizado,igpm,ipca,selic,month
0,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,2023-01-04,ABCB4,14.19,8.75,1.22,0.000,0.0334,0.000,0.00,0.00,0.00,0.00,-0.000000,0.000000,0.0000,0.1389,0.00,0.198734,0.104159,0.00,0.623592,2013-01-05,2023,0.202566,2.188247e+07,1.573185e+10,-0.245499,0.136948,2013,2012-12-31,0.6,7.3,1.9843,2.6981,202662.4,1656.7,0.34,0.86,0.02726,1
1,AGRO3,ON NM,BRASILAGRO ON NM,Agropecuária,Agricultura,2023-01-04,AGRO3,10.09,-22.64,1.06,3.842,0.0000,0.808,5.25,-28.57,16.14,-30.41,0.347950,-0.609352,-0.0327,-0.0468,2.16,0.171306,0.687695,0.16,1.370918,2013-01-05,2023,0.288175,2.133847e+06,8.092218e+08,-0.386550,0.543067,2013,2012-12-31,0.6,7.3,1.9843,2.6981,202662.4,1656.7,0.34,0.86,0.02726,1
2,ALPA3,ON N1,ALPARGATAS ON N1,"Tecidos, Vestuário e Calçados",Calçados,2023-01-04,ALPA3,14.95,21.33,3.60,2.033,0.0142,2.107,5.44,18.14,7.73,17.01,1.832448,6.775139,0.1895,0.1686,2.30,1.871477,3.179211,0.19,2.004121,2013-01-05,2023,0.014096,4.858312e+05,5.084878e+08,0.061175,0.082829,2013,2012-12-31,0.6,7.3,1.9843,2.6981,202662.4,1656.7,0.34,0.86,0.02726,1
3,ALPA4,PN N1,ALPARGATAS PN N1,"Tecidos, Vestuário e Calçados",Calçados,2023-01-04,ALPA4,15.15,21.62,3.64,2.060,0.0154,2.136,5.51,18.38,7.84,17.25,1.832448,6.775139,0.1895,0.1686,2.30,11.222601,3.179211,0.19,2.004121,2013-01-05,2023,0.014096,4.858312e+05,5.084878e+08,0.061175,0.082829,2013,2012-12-31,0.6,7.3,1.9843,2.6981,202662.4,1656.7,0.34,0.86,0.02726,1
4,AZEV3,ON,AZEVEDO & TRAVASSOS ON,Construção e Engenharia,Construção Pesada,2023-01-04,AZEV3,10.01,39.69,8.05,1.207,0.0000,1.891,6.00,21.78,-12.97,22.87,0.438720,0.549261,0.0961,0.2028,2.04,0.000000,0.821983,0.52,-0.501056,2013-01-05,2023,0.055347,4.233654e+05,4.084512e+07,0.126276,0.140703,2013,2012-12-31,0.6,7.3,1.9843,2.6981,202662.4,1656.7,0.34,0.86,0.02726,1


In [41]:
tickers = database_full['Papel'].unique()

In [42]:
def forecast_model(vals):

    try:
        forecast_model = SimpleExpSmoothing(vals).fit(
        smoothing_level=0.75, optimized=False
        )
        return forecast_model.forecast(1)
       
    except:
        print('forecast fail')
        return 0 

In [43]:
def get_variations(data, windows, stock_info_cols):
    """
    Compute various statistics for a set of rolling windows of data.

    Parameters
    ----------
    data : pandas.DataFrame
        The input DataFrame containing the data to compute statistics for.
    windows : list of list of int
        A list of rolling windows to compute statistics for. Each window should be a list of years.
    stock_info_cols : list of str
        The names of the columns containing stock information.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the computed statistics for each rolling window.
    """

    data_line_full = pd.DataFrame()
    cols = ['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg',
            'pebit', 'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc',
            'liq2m', 'patrliq', 'divbpatr', 'c5y',
            'preco_do_petroleo','dolar_comercial','euro','ibovespa','pib_dolarizado','igpm','ipca','selic']
    extra_cols = list(data.columns[~np.isin(data.columns, stock_info_cols + cols)])
    
    for window in windows:
        window.sort(reverse=True)

        data_window = data[np.isin(data['year'],window)]
        data_window = data_window.sort_values(by='year', ascending=False)
        # data_full_window = data_full[np.isin(data_full['year'],window)]
        # data_full_window = data_full_window.sort_values(by='year', ascending=False)

        # rel_base = relativize(data_full_window, cols_to_rel, 'Setor')

        if np.sum(data['year']>np.max(window)):

            data_line = {}

            #get label
            data_line['dy_label'] = data[data['year'] == (np.max(window)+1)]['dy'].values[0]

            for col in cols:
                data_line[col] = data_window[col].iloc[0]
                try:
                    data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
                except:
                    data_line[f'{col}_rel'] = 0

                data_line[f'{col}_median'] = data_window[col].median()

                data_line[f'{col}_mean'] = data_window[col].mean()

                try:
                    data_line[f'{col}_mean_median_ratio'] = data_line[f'{col}_mean']/data_line[f'{col}_median']
                except:
                    data_line[f'{col}_rel'] = 0

                data_line[f'{col}_std'] = data_window[col].std()

                try:
                    data_line[f'{col}_spread'] = (data_window[col].max() - data_window[col].min()) / data_window[col].mean()
                except:
                    data_line[f'{col}_spread'] = 0

                try:
                    data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
                except:
                    data_line[f'{col}_spread_rel_now'] = 0

                if col == 'dy':
                    vals = data_window[col].fillna(0).values

                    data_line[f'{col}_sequence'] = np.sum(vals > 0)

                    data_line[f'{col}_sequence_bool'] = int(np.sum(vals > 0) == len(vals))

                    data_line[f'{col}_forecast'] = forecast_model(vals)

            for extra_col in extra_cols:
                data_line[extra_col] = data_window[extra_col].iloc[0]
           
            df_data_line = pd.DataFrame(data_line, index=[0])
            data_line_full = pd.concat([data_line_full, df_data_line])

        
        else:
            continue
    
    statistic_cols = list(data_line_full.columns)
    # extra_cols = list(data.columns[~np.isin(data.columns, stock_info_cols + cols)])

    data_line_full[stock_info_cols] = data[stock_info].iloc[0]
    data_line_full = data_line_full[stock_info_cols + statistic_cols]

    return data_line_full

In [44]:
def get_full_variations(data, tickers, window, stock_info_cols):

    df_variations = pd.DataFrame()

    for ticker in tickers:

        data_ticker = data[data['Papel']==ticker].sort_values('year', ascending=False).copy()

        max = data_ticker['year'].max()
        min = data_ticker['year'].min()

        if (max - min) < window:
            continue
        else:
            windows = get_windows(max, min, window=window, step=1)

            ticker_variation = get_variations(data_ticker, windows, stock_info_cols)
        
            df_variations = pd.concat([df_variations, ticker_variation])

    return df_variations


In [45]:
base_dataset = get_full_variations(database_full, tickers, window=5, stock_info_cols=stock_info)

C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\2055789708.py:47: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\2055789708.py:68: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\2055789708.py:47: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\2055789708.py:68: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_spread_rel_now'] = data_line[f'{col}_spread']/data_line[col]
C:\Users\Pontello\AppData\Local\Temp\ipykernel_4356\2055789708.py:47: RuntimeWarning: invalid value encountered in double_scalars
  data_line[f'{col}_rel'] 

In [46]:
base_dataset.head()

,Papel,Tipo,Empresa,Setor,Subsetor,dy_label,cotacao,cotacao_rel,cotacao_median,cotacao_mean,cotacao_mean_median_ratio,cotacao_std,cotacao_spread,cotacao_spread_rel_now,pl,pl_rel,pl_median,pl_mean,pl_mean_median_ratio,pl_std,pl_spread,pl_spread_rel_now,pvp,pvp_rel,pvp_median,pvp_mean,pvp_mean_median_ratio,pvp_std,pvp_spread,pvp_spread_rel_now,psr,psr_rel,psr_median,psr_mean,psr_std,psr_spread,psr_spread_rel_now,dy,dy_rel,dy_median,dy_mean,dy_mean_median_ratio,dy_std,dy_spread,dy_spread_rel_now,dy_sequence,dy_sequence_bool,dy_forecast,pa,pa_rel,pa_median,pa_mean,pa_std,pa_spread,pa_spread_rel_now,pcg,pcg_rel,pcg_median,pcg_mean,pcg_std,pcg_spread,pcg_spread_rel_now,pebit,pebit_rel,pebit_median,pebit_mean,pebit_std,pebit_spread,pebit_spread_rel_now,pacl,pacl_rel,pacl_median,pacl_mean,pacl_std,pacl_spread,pacl_spread_rel_now,evebit,evebit_rel,evebit_median,evebit_mean,evebit_std,evebit_spread,evebit_spread_rel_now,mrgebit,mrgebit_rel,mrgebit_median,mrgebit_mean,mrgebit_std,mrgebit_spread,mrgebit_spread_rel_now,mrgliq,mrgliq_rel,mrgliq_median,mrgliq_mean,mrgliq_std,mrgliq_spread,mrgliq_spread_rel_now,roic,roic_rel,roic_median,...,divbpatr_mean,divbpatr_std,divbpatr_spread,divbpatr_spread_rel_now,c5y,c5y_rel,c5y_median,c5y_mean,c5y_mean_median_ratio,c5y_std,c5y_spread,c5y_spread_rel_now,preco_do_petroleo,preco_do_petroleo_rel,preco_do_petroleo_median,preco_do_petroleo_mean,preco_do_petroleo_mean_median_ratio,preco_do_petroleo_std,preco_do_petroleo_spread,preco_do_petroleo_spread_rel_now,dolar_comercial,dolar_comercial_rel,dolar_comercial_median,dolar_comercial_mean,dolar_comercial_mean_median_ratio,dolar_comercial_std,dolar_comercial_spread,dolar_comercial_spread_rel_now,euro,euro_rel,euro_median,euro_mean,euro_mean_median_ratio,euro_std,euro_spread,euro_spread_rel_now,ibovespa,ibovespa_rel,ibovespa_median,ibovespa_mean,ibovespa_mean_median_ratio,ibovespa_std,ibovespa_spread,ibovespa_spread_rel_now,pib_dolarizado,pib_dolarizado_rel,pib_dolarizado_median,pib_dolarizado_mean,pib_dolarizado_mean_median_ratio,pib_dolarizado_std,pib_dolarizado_spread,pib_dolarizado_spread_rel_now,igpm,igpm_rel,igpm_median,igpm_mean,igpm_mean_median_ratio,igpm_std,igpm_spread,igpm_spread_rel_now,ipca,ipca_rel,ipca_median,ipca_mean,ipca_mean_median_ratio,ipca_std,ipca_spread,ipca_spread_rel_now,selic,selic_rel,selic_median,selic_mean,selic_mean_median_ratio,selic_std,selic_spread,selic_spread_rel_now,Data_ult_cot,papel,date,Data_ult_cot_year,mrgliq_field,liq2m_field,patrliq_field,mrgebit_field,c5y_field,year,data,indice_da_industria,month,psr_mean_median_ratio,pa_mean_median_ratio,pcg_mean_median_ratio,pebit_mean_median_ratio,pacl_mean_median_ratio,evebit_mean_median_ratio,mrgebit_mean_median_ratio,mrgliq_mean_median_ratio,roic_mean_median_ratio,liqc_mean_median_ratio,divbpatr_mean_median_ratio
0,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,0.0308,16.60,1.169838,13.220,13.505000,1.021558,3.690056,0.761940,0.045900,7.83,0.894857,7.50,7.005000,0.934000,1.872173,0.705211,0.090065,1.01,0.827869,0.990,0.965000,0.974747,0.251456,0.704663,0.697686,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0643,1.925150,0.05205,0.050983,0.979507,0.010434,0.606080,9.425823,6.0,1.0,0.037529,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.0,0.0,-0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,7.652507,12.271654,0.649625,1.746972,2.689199,2.899390,4.224468,0.552037,0.58,0.966667,0.895,0.853333,0.953445,0.221420,0.597656,1.030442,3.1730,1.599053,3.15175,2.953567,0.937120,0.726164,0.681989,0.214935,3.9482,1.463326,3.39130,3.485117,1.027664,0.591138,0.476655,0.120727,246444.2,1.216033,207967.50,216234.550000,1.039752,18490.953894,0.207416,8.416365e-07,1587.2,0.958049,1607.35,1605.233333,0.998683,120.637053,0.215296,0.000136,0.76,2.235294,0.7,0.686667,0.980952,0.276164,1.165049,1.532959,0.29,0.337209,0.705,0.765000,1.085106,0.426556,1.281046,4.417399,0.02648

In [47]:
base_dataset[[col for col in base_dataset.columns if 'dy' in col]]

,dy_label,dy,dy_rel,dy_median,dy_mean,dy_mean_median_ratio,dy_std,dy_spread,dy_spread_rel_now,dy_sequence,dy_sequence_bool,dy_forecast
0,0.0308,0.0643,1.925150,0.05205,0.050983,0.979507,0.010434,0.606080,9.425823,6.0,1.0,0.037529
0,0.0466,0.0308,0.636364,0.05205,0.050550,0.971182,0.011326,0.662710,21.516565,6.0,1.0,0.049884
0,0.0322,0.0466,0.836625,0.05205,0.050250,0.965418,0.011418,0.666667,14.306152,6.0,1.0,0.054350
0,0.0581,0.0322,0.665289,0.04750,0.046333,0.975439,0.013084,0.723022,22.454086,6.0,1.0,0.050287
0,0.0849,0.0581,1.043088,0.05115,0.047950,0.937439,0.013960,0.698644,12.024861,6.0,1.0,0.055972
...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0328,0.0169,16.900000,0.01605,0.018117,1.128764,0.013853,2.345906,138.811016,6.0,1.0,0.009960
0,0.0011,0.0000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000
0,0.0000,0.0000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000
0,0.0000,0.0000,0.000000,0.00000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000


In [48]:
columns_to_drop = ['cotacao','cotacao_median', 'cotacao_mean']
base_dataset.drop(columns_to_drop,axis=1, inplace=True)

In [49]:
def inpute_median_by_category(data,group_by_col):

    cols_with_nans = data.columns[data.isna().any()].tolist()

    for col in cols_with_nans:
       data[col] = data.groupby(group_by_col)[col].transform(lambda x: x.fillna(x.median()))

    return data


In [50]:
inpute_median_by_category(base_dataset,'Setor').isna().sum().sort_values(ascending=False)

divbpatr_mean_median_ratio    30
divbpatr_spread_rel_now       30
dy_mean_median_ratio          11
dy_spread_rel_now             10
mrgebit_rel                    1
                              ..
mrgebit_std                    0
mrgebit_spread                 0
mrgebit_spread_rel_now         0
mrgliq                         0
liq2m_rel                      0
Length: 235, dtype: int64

In [51]:
base_dataset = inpute_median_by_category(base_dataset,'Setor')
# base_dataset.fillna(np.median, inplace=True)
base_dataset.isna().sum().sort_values(ascending=False)[:20]



divbpatr_mean_median_ratio           30
divbpatr_spread_rel_now              30
dy_mean_median_ratio                 11
dy_spread_rel_now                    10
mrgebit_rel                           1
c5y_rel                               1
mrgliq_rel                            1
dolar_comercial_std                   0
preco_do_petroleo_std                 0
preco_do_petroleo_spread              0
preco_do_petroleo_spread_rel_now      0
dolar_comercial                       0
dolar_comercial_rel                   0
dolar_comercial_median                0
dolar_comercial_mean                  0
dolar_comercial_mean_median_ratio     0
dolar_comercial_spread                0
dolar_comercial_spread_rel_now        0
euro                                  0
euro_rel                              0
dtype: int64

In [52]:
# base_dataset = base_dataset.drop(columns_to_drop, axis=1)
base_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
base_dataset = base_dataset.dropna()
base_dataset.isna().sum().sort_values(ascending=False)[:20]

Papel                                  0
dolar_comercial_spread                 0
preco_do_petroleo_median               0
preco_do_petroleo_mean                 0
preco_do_petroleo_mean_median_ratio    0
preco_do_petroleo_std                  0
preco_do_petroleo_spread               0
preco_do_petroleo_spread_rel_now       0
dolar_comercial                        0
dolar_comercial_rel                    0
dolar_comercial_median                 0
dolar_comercial_mean                   0
dolar_comercial_mean_median_ratio      0
dolar_comercial_std                    0
dolar_comercial_spread_rel_now         0
ibovespa_std                           0
euro                                   0
euro_rel                               0
euro_median                            0
euro_mean                              0
dtype: int64

### Storing refined data into Google Cloud

In [53]:
client, bucket = initialize_bucket(credentials_path,'storage-barsianize')

# upload base dataset to Google Cloud Storage
blob = bucket.blob('refined/base_dataset.csv')
blob.upload_from_string(base_dataset.to_csv(), 'text/csv',)

> To Dos
* pegar primeiras 23 features e pensar em feature engineering
    * relativizar os valores absolutos (mrgliq, Lucro, patrliq, divbrut, ativo_circulante, receita_liquida, etc)
    * testar outras formas de scaling
    * spread relativo sempre ao now

